In [10]:
from stable_baselines3 import PPO
from stable_baselines3.ppo.policies import MlpPolicy
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.env_checker import check_env
import gym
import numpy as np
from RL import helpers
from Hack import load

In [2]:
epex = load.epex().load()
price_array = epex['apx_da_hourly'].values


In [14]:
env = helpers.energy_price_env(price_array, max_time=2*24*28)
model = PPO(MlpPolicy, env, verbose=1)
check_env(env, warn=True)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


C:\Users\Ronan\Anaconda3\envs\ml\lib\site-packages\gym\logger.py:34: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))


In [15]:
mean_reward_before_train = helpers.evaluate(model, num_episodes=1)

Mean reward: -40.48573 Num episodes: 10


In [16]:
# Train the agent for 10000 steps
model.learn(total_timesteps=10000)

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1.34e+03 |
|    ep_rew_mean     | 118      |
| time/              |          |
|    fps             | 1159     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.34e+03    |
|    ep_rew_mean          | 64.4        |
| time/                   |             |
|    fps                  | 864         |
|    iterations           | 2           |
|    time_elapsed         | 4           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.006829649 |
|    clip_fraction        | 0.0312      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.09       |
|    explained_variance   | -0.0167     |
|    learning_rate        | 0.

In [20]:
# Trained Agent, after training
periods = len(price_array)-1  # 4 * 28*24*2
new_env =  DummyVecEnv([lambda: helpers.energy_price_env(price_array, start_time=0, max_time = periods)])
mean_reward_after_train = helpers.evaluate(model, new_env=new_env, num_episodes=1, x_index="Time")

C:\Users\Ronan\Anaconda3\envs\ml\lib\site-packages\gym\logger.py:34: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))


Mean reward: 3595.2332 Num episodes: 1
